<a href="https://colab.research.google.com/github/FeelOneE/price_prediction/blob/main/coin_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/스터디/코인 트레이드/data


/content/drive/MyDrive/스터디/코인 트레이드/data


In [3]:
%ls -al

total 3758
-rw------- 1 root root 3847495 Dec 18 08:07 XRP.csv


In [24]:
import pandas as pd

data = pd.read_csv("XRP.csv")
data.head(5)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2021-12-17T15:05:00,2021-12-18T00:05:00,986.0,987.0,984.0,987.0,1639753799934,1.785798e+09,1.812305e+06,5
1,KRW-XRP,2021-12-17T15:00:00,2021-12-18T00:00:00,989.0,989.0,981.0,986.0,1639753500090,2.128017e+09,2.161762e+06,5
2,KRW-XRP,2021-12-17T14:55:00,2021-12-17T23:55:00,984.0,989.0,983.0,989.0,1639753198185,2.275873e+09,2.311628e+06,5
3,KRW-XRP,2021-12-17T14:50:00,2021-12-17T23:50:00,987.0,990.0,984.0,985.0,1639752896408,3.238981e+09,3.281353e+06,5
4,KRW-XRP,2021-12-17T14:45:00,2021-12-17T23:45:00,983.0,987.0,981.0,987.0,1639752599653,5.052325e+09,5.141900e+06,5


In [25]:
# 시간 T를 기준으로 나누어 날짜는 제외하고 시간 분만 남게 처리
data['candle_date_time_kst'] = data.candle_date_time_kst.str.split('T', expand=True).apply(lambda x : x[1], axis=1)
data.head(5)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2021-12-17T15:05:00,00:05:00,986.0,987.0,984.0,987.0,1639753799934,1.785798e+09,1.812305e+06,5
1,KRW-XRP,2021-12-17T15:00:00,00:00:00,989.0,989.0,981.0,986.0,1639753500090,2.128017e+09,2.161762e+06,5
2,KRW-XRP,2021-12-17T14:55:00,23:55:00,984.0,989.0,983.0,989.0,1639753198185,2.275873e+09,2.311628e+06,5
3,KRW-XRP,2021-12-17T14:50:00,23:50:00,987.0,990.0,984.0,985.0,1639752896408,3.238981e+09,3.281353e+06,5
4,KRW-XRP,2021-12-17T14:45:00,23:45:00,983.0,987.0,981.0,987.0,1639752599653,5.052325e+09,5.141900e+06,5


In [26]:
# 시간을 분으로 치환
data['candle_date_time_kst'] = data.candle_date_time_kst.str.split(':', expand=True).apply(lambda x : (int(x[0])*60)+(int(x[1])) , axis=1)
data.head(5)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2021-12-17T15:05:00,5,986.0,987.0,984.0,987.0,1639753799934,1.785798e+09,1.812305e+06,5
1,KRW-XRP,2021-12-17T15:00:00,0,989.0,989.0,981.0,986.0,1639753500090,2.128017e+09,2.161762e+06,5
2,KRW-XRP,2021-12-17T14:55:00,1435,984.0,989.0,983.0,989.0,1639753198185,2.275873e+09,2.311628e+06,5
3,KRW-XRP,2021-12-17T14:50:00,1430,987.0,990.0,984.0,985.0,1639752896408,3.238981e+09,3.281353e+06,5
4,KRW-XRP,2021-12-17T14:45:00,1425,983.0,987.0,981.0,987.0,1639752599653,5.052325e+09,5.141900e+06,5


In [27]:
# 안쓰는 컬럼 삭제 market, utc_time, unit, timestamp
data.drop(['market','candle_date_time_utc','unit','timestamp'], axis=1, inplace=True)
data.head(5)

,candle_date_time_kst,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume
0,5,986.0,987.0,984.0,987.0,1.785798e+09,1.812305e+06
1,0,989.0,989.0,981.0,986.0,2.128017e+09,2.161762e+06
2,1435,984.0,989.0,983.0,989.0,2.275873e+09,2.311628e+06
3,1430,987.0,990.0,984.0,985.0,3.238981e+09,3.281353e+06
4,1425,983.0,987.0,981.0,987.0,5.052325e+09,5.141900e+06


In [28]:
# price, volume 소수점 5자리까지만 
data['candle_acc_trade_price'] = data.candle_acc_trade_price.apply(lambda x : '%.5f' % x)
data['candle_acc_trade_volume'] = data.candle_acc_trade_volume.apply(lambda x : '%.5f' % x)
data.head(5)

,candle_date_time_kst,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume
0,5,986.0,987.0,984.0,987.0,1785797775.07049,1812305.45733
1,0,989.0,989.0,981.0,986.0,2128016988.39383,2161761.89739
2,1435,984.0,989.0,983.0,989.0,2275873191.28294,2311628.47649
3,1430,987.0,990.0,984.0,985.0,3238981389.51325,3281353.39430
4,1425,983.0,987.0,981.0,987.0,5052324608.41463,5141900.24199


In [33]:
# 시간순서대로 데이터 다시 정렬
data = data[::-1]
data.head(5)

,candle_date_time_kst,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume
30199,1240,1475.0,1480.0,1470.0,1480.0,2449703432.48617,1661509.79377
30198,1245,1480.0,1480.0,1470.0,1480.0,1401303636.71963,949623.96941
30197,1250,1475.0,1480.0,1475.0,1480.0,964682189.53982,652588.80397
30196,1255,1480.0,1480.0,1475.0,1475.0,685879546.75352,464264.68260
30195,1260,1475.0,1480.0,1475.0,1480.0,519939889.71916,351858.39885


In [81]:
# 정답 라벨 붙이기
# 데이터를 60개로 묶고, 마지막 종가(trade_price)를 기준으로 
# 다음 60개의 데이터 중 최대 값이 종가의 1.035배 이상인 적이 있는지 찾기

def find_label(index):
  sample = data[index:index+60]
  next_sample = data[index+60:index+120]

  trade_price = sample['trade_price'].iloc[-1]
  #print(trade_price)

  price_list = []
  price_list.extend(next_sample['opening_price'])
  price_list.extend(next_sample['high_price'])
  price_list.extend(next_sample['low_price'])
  price_list.extend(next_sample['trade_price'])

  max_price = max(price_list)
  #print(max_price)

  expect_price = trade_price*1.035
  #print(expect_price)

  if max_price >= expect_price :
    return 1
    # print(trade_price)
    # print(expect_price)
    # print(max_price)
    # print("find label! : ", index)
    # print('-'*30)
  return 0

In [84]:
# 전체 데이터와 정답 레이블의 비율
total_cnt = 0
cnt = 0
skip_size = 1
for idx in range(0, len(data)-120, skip_size):
  cnt += find_label(idx)
  total_cnt += 1

rate = float(cnt / total_cnt)
print(rate)


0.08939494680851064


# 모델링

In [ ]:
# input shape 2d = (데이터 개수, 하나의 데이터의 행 수, 열 수)


In [31]:
# 데이터 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
std_data = data.copy()
std_data[:] = scaler.fit_transform(data[:])
std_data.head(20)

,candle_date_time_kst,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume
0,0.003484,0.081897,0.070809,0.168627,0.083333,0.016583,0.023751
1,0.000000,0.086207,0.073699,0.164706,0.081897,0.019762,0.028332
2,1.000000,0.079023,0.073699,0.167320,0.086207,0.021135,0.030297
3,0.996516,0.083333,0.075145,0.168627,0.080460,0.030081,0.043008
4,0.993031,0.077586,0.070809,0.164706,0.083333,0.046925,0.067397
5,0.989547,0.080460,0.070809,0.166013,0.077586,0.026309,0.037720
6,0.986063,0.090517,0.078035,0.167320,0.079023,0.068287,0.097685
7,0.982578,0.091954,0.079480,0.176471,0.090517,0.020885,0.029732
8,0.979094,0.093391,0.082370,0.179085,0.091954,0.017311,0.024588
9,0.975610,0.090517,0.080925,0.177778,0.093391,0.015538,0.022096
